<a href="https://colab.research.google.com/github/AlanKev117/data-engineering-bootcamp/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project for Wizeline's Data Engineering Bootcamp
---
### By Alan Fuentes

In [58]:
import pandas as pd

## A quick review through the sample dataset
I decided to take a look at the sample dataset to get familiar with the column names and also to choose the ones that might be useful in order to find some interesting information

In [ ]:
sample = pd.read_csv("sample.csv")
sample.head(50)

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,UTILES ESCOLARES,25.90,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,27.50,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,13.90,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
3,COLORES DE MADERA,CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646,PINCELIN,MATERIAL ESCOLAR,UTILES ESCOLARES,46.90,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
4,COLOR LARGO,CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,115.00,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
5,BOLIGRAFO,BLISTER 3 PIEZAS. PUNTO FINO. GEL,BIC. CRISTAL GEL,MATERIAL ESCOLAR,UTILES ESCOLARES,32.50,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
6,CINTA ADHESIVA,BOLSA 1 PIEZA. 12 MM. X 33 M. C.B. 100317,SCOTCH 3M. 600,MATERIAL ESCOLAR,UTILES ESCOLARES,9.00,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
7,COLORES DE MADERA,CAJA 24 PIEZAS. LARGO. GRATIS 1 SACAPUNTAS. C....,KORES. KOLORES,MATERIAL ESCOLAR,UTILES ESCOLARES,95.90,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
8,CRAYONES,CAJA 24 CERAS. TAMANO REGULAR. C.B. 608394,PAPER MATE. CARMEN,MATERIAL ESCOLAR,UTILES ESCOLARES,23.20,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
9,PAN BLANCO BOLILLO,PIEZA,S/M,PAN,BASICOS,1.20,2011-01-10 00:00:00.000,COMERCIAL MEXICANA,TIENDA DE AUTOSERVICIO,COMERCIAL MEXICANA SUCURSAL VILLAS DE LA HACIENDA,AV. LAGO DE GUADALUPE S/N ESQ. DE LAS VILLAS,MÉXICO,ATIZAPAN,NaN,NaN


I chose to ignore **latitude**, **longitude**, **commercial name** and **registry date** since they do not seem to provide useful information.
I also took out **category**, **town** and **address** since they are more suitable for a deeper analysis.

In [ ]:
columns_to_use = sample.columns.to_list()
columns_to_remove = ["latitud", "longitud", "nombreComercial", "fechaRegistro", "categoria", "municipio", "direccion"]
columns_to_use = [column for column in columns_to_use if column not in columns_to_remove]
columns_to_use

['producto',
 'presentacion',
 'marca',
 'catalogo',
 'precio',
 'cadenaComercial',
 'giro',
 'estado']

I created a symbolic link to the large file into my Drive account so I could read it faster in Google Colab.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


I performed some testing to find a good chunk size.

In [ ]:
CHUNK_SIZE = 1e4
HUGE_FILE_PATH = "/content/drive/MyDrive/profeco.zip"

In [ ]:
state_and_product_columns = ["estado", "producto", "presentacion"]
commercial_chain_columns = ["cadenaComercial"]
aggregate_frame_column = "producto"

In [ ]:
data = pd.read_csv(HUGE_FILE_PATH, chunksize=CHUNK_SIZE, usecols=columns_to_use)

In order to save memory, I processed the data chunks as they were read. The snippet below groups and counts data in the chunks as required and stores the resulting set into a bigger one for further consulting. 

In [ ]:
chunk = data.get_chunk()
items_counted_by_state_and_product = chunk.groupby(state_and_product_columns)[aggregate_frame_column].count()
items_counted_by_commercial_chain = chunk.groupby(commercial_chain_columns)[aggregate_frame_column].count()
#cheapest_brands_by_state
#while True:
for i in range(3):
    try:
        chunk = data.get_chunk()
    except StopIteration:
        print("Dataset read")
        break
    state_product_count = chunk.groupby(state_and_product_columns)[aggregate_frame_column].count()
    commercial_chain_count = chunk.groupby(commercial_chain_columns)[aggregate_frame_column].count()
    items_counted_by_state_and_product = items_counted_by_state_and_product.add(state_product_count, fill_value=0)
    items_counted_by_commercial_chain = items_counted_by_commercial_chain.add(commercial_chain_count, fill_value=0)

## A1: How many commercial chains are monitored, and therefore, included in this database?

Since the Series `items_counted_by_commercial_chain` has already an index whose keys are the commercial chains, we only need to find the length of that as a list.

In [ ]:
commercial_chains = items_counted_by_commercial_chain.index.to_list()
print(f"There are {len(commercial_chains)} commercial chains whose products are being monitored")
print(f"They are: {commercial_chains}")

There are 79 commercial chains whose products are being monitored
They are: ['7 ELEVEN', 'ABARROTERA DE TLAXCALA', 'ABARROTERA GUADALUPANA (FRUTAS)', 'ABARROTES LA VIOLETA', 'ALMACENES ZARAGOZA', 'ALPRECIO', 'APARATOS Y MUEBLES DE MORELIA', 'AUTOSERVICIO GUTIERREZ RIZO', 'AUTOSERVICIO ZARAGOZA', 'BENAVIDES', 'BODEGA AURRERA', 'BODEGA COMERCIAL MEXICANA', 'CASA LEY', 'CHEDRAUI', 'COMERCIAL MEXICANA', 'COPPEL', 'DROGUERIA MEDINA', 'EL PITICO', 'ELECTROMUEBLES', 'ELEKTRA', 'FAMSA', 'FARMACIA', 'FARMACIA ABC', 'FARMACIA DE SIMILARES', 'FARMACIA DURANGO', 'FARMACIA ESPECIALIZADA', 'FARMACIA GUADALAJARA', 'FARMACIA IDEAL', 'FARMACIA ISSEG', 'FARMACIA LA LUZ', 'FARMACIA LA UNION', 'FARMACIA MODERNA', 'FARMACIA PARIS', 'FARMACIA POPULAR ROMAN', 'FARMACIA TORRE MEDICA', 'FARMACIA UNIVERSITARIA ALEXANDER FLEMING', 'FARMACIAS EL FENIX', 'FARMACIAS SIMILARES', 'GIGANTE', 'H.E.B.', 'HERMANOS VAZQUEZ', 'HIPERMERCADO SORIANA', 'I.M.S.S.', 'I.S.S.S.T.E.', 'ISSSTEY', 'LEY (AUTOSERVICIO)', 'LIVERPOOL', 

## A2: Top 10 monitored products by State
The Series `items_counted_by_state_and_product` has grouped a count of entries grouped by state and product name. Then, it is possible to filter the 10 most counted items grouped by State.

In [ ]:
top_10_by_state = items_counted_by_state_and_product.groupby("estado", group_keys=False).nlargest(10)
print("Top ten monitored products by State:")
top_10_by_state.head(50)

Top ten monitored products by State:


estado                producto                  presentacion                                                                       marca                                      
AGUASCALIENTES        TORTILLA DE MAIZ          1 KG. GRANEL                                                                       S/M                                             5.0
                      HORNO DE MICROONDAS       XO 1410 MD. 1.4 PIES CUBICOS                                                       MABE                                            4.0
                      LAVADORAS                 7 MWT 99900. 16 KG. AGITADOR. CENTRIFUGADO                                         WHIRLPOOL                                       4.0
                      CAMARAS DIGITALES         DMC-FH3. MEGA PIXELES 14. ZOOM OPTICO 5 X                                          PANASONIC                                       3.0
                                                DSC-W320. MEGA PIXELES 14.1. ZOOM OPTICO 4 X 

## A3: Commercial chain with the most monitored products
It reduces to finding the row with highest value in `items_counted_by_commercial_chain`.

In [ ]:
print("Commercial chain with the most monitored products:")
print(items_counted_by_commercial_chain.nlargest(1))

Commercial chain with the most monitored products:
cadenaComercial
COMERCIAL MEXICANA    1508.0
Name: producto, dtype: float64


## A4: An interesting fact: 
